# 1 ASOS_DATA PROCESSING

In [2]:
# Basic setting for Jupyter_notebook to import utils
import os
import sys

notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, "../../"))

sys.path.append(project_root)

In [66]:
import os
import sys
import pandas as pd
from utils import folder_utils,time_utils
from tqdm import tqdm

In [47]:
def extract_data_to_df(country, data_folder, data_category, output_folder):
    # Specify the folder path
    input_folder_path = folder_utils.find_folder(
        country, data_folder, data_category, output_folder
    )
    # Initialize a dataframe to store all the data from the same country
    raw_df = pd.DataFrame()
    # Get the filenames of all CSV files under the folder except the station network file
    csv_files = [
        f
        for f in os.listdir(input_folder_path)
        if f.endswith('.csv') and "asos_station_network" not in f
    ]
    # Read and merge the csv files in queue
    for csv_file in tqdm(csv_files):
        csv_file_path = os.path.join(input_folder_path, csv_file)
        df = pd.read_csv(csv_file_path)
        raw_df = pd.concat([raw_df, df], ignore_index=True)

    return raw_df

In [48]:
def process_asos_rawdata(df):

    
    # Split "valid" column into "date" and "time" columns
    df['date'] = (
        df['valid'].str.split(' ', expand=True)[0].str.replace('-', '').astype(int)
    )
    df['time'] = df['valid'].str.split(' ', expand=True)[1]

    # Convert Fahrenheit to Kelvin for "tmpf", "feel" and "dwpf" columns
    df['tmpf'] = (df['tmpf'] - 32) * 5 / 9 + 273.15
    df['tmpf'] = df['tmpf'].round(1)

    df['dwpf'] = (df['dwpf'] - 32) * 5 / 9 + 273.15
    df['dwpf'] = df['dwpf'].round(1)

    # Convert Fahrenheit to Kelvin for "feel" column
    df['feel'] = (df['feel'] - 32) * 5 / 9 + 273.15
    df['feel'] = df['feel'].round(1)
    
    # Convert knots to m/s for "sknt" and "gust" columns
    df['sknt'] = df['sknt'] * 0.514444
    df['gust'] = df['gust'] * 0.514444

    # Convert inches to meters for "p01i" and "alti" columns
    df['p01i'] = pd.to_numeric(df['p01i'], errors='coerce') * 0.0254
    df['alti'] = df['alti'] * 0.0254

    # Convert millibar to Pa for "mslp" column
    df['mslp'] = df['mslp'] * 100

    # Drop columns
    columns_to_drop = [
        'lon',
        'lat',
        'elevation',
        'valid',
        'skyc1',
        'skyc2',
        'skyc3',
        'skyc4',
        'skyl1',
        'skyl2',
        'skyl3',
        'skyl4',
        'wxcodes',
        'ice_accretion_1hr',
        'ice_accretion_3hr',
        'ice_accretion_6hr',
        'peak_wind_gust',
        'peak_wind_drct',
        'peak_wind_time',
        'metar',
        'snowdepth',
    ]
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    return df

In [49]:
def save_asos_processed_data(
    processed_df, country, data_folder, data_category, output_folder
):
    output_directory = folder_utils.create_folder(
        country, data_folder, data_category, output_folder
    )
    output_filename = f"{country}_ASOS_processed_data.csv"
    output_filepath = os.path.join(output_directory, output_filename)
    processed_df.to_csv(output_filepath, index=False, encoding="utf-8")
    print(f'{output_filename} done!')

In [50]:
# Example usage

country = "GB"
data_folder = "data"
data_test_category = "test_data"
data_read_category = "raw_data"
data_save_category = "processed_data"
output_folder = "ASOS_DATA"

In [51]:
raw_df = extract_data_to_df(country, data_folder, data_read_category , output_folder)

  3%|█▎                                               | 3/112 [00:00<00:10, 10.84it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
  4%|██▏                                              | 5/112 [00:00<00:11,  9.60it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
  6%|███                                              | 7/112 [00:00<00:08, 12.02it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
 10%|████▋                                           | 11/112 [00:00<00:08, 11.66it/s]/var/

In [52]:
raw_df

,station,valid,lon,lat,elevation,tmpf,dwpf,relh,drct,sknt,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,EGUN,2022-01-01 00:26,0.4864,52.3619,10.0,53.6,53.6,100.00,210.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.6,EGUN 010026Z AUTO 21009KT 9999 BKN013 12/12 A3...,NaN
1,EGUN,2022-01-01 00:56,0.4864,52.3619,10.0,54.0,54.0,100.00,210.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,EGUN 010056Z AUTO 21010KT 9999 SCT012 BKN035 1...,NaN
2,EGUN,2022-01-01 01:26,0.4864,52.3619,10.0,55.4,55.4,100.00,200.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.4,EGUN 010126Z AUTO 20009KT 9999 SCT034 13/13 A3...,NaN
3,EGUN,2022-01-01 01:56,0.4864,52.3619,10.0,54.7,54.7,100.00,210.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.7,EGUN 010156Z AUTO 21011KT 9999 CLR 13/13 A3008...,NaN
4,EGUN,2022-01-01 02:56,0.4864,52.3619,10.0,55.6,55.6,100.00,200.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.6,EGUN 010256Z AUTO 20009KT 9999 CLR 13/13 A3007...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,2023-08-01 19:20,0.0325,51.3308,183.0,60.8,57.2,87.92,210.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.8,EGKB 011920Z 21011KT 170V240 9999 FEW015 16/14...,NaN
2038807,EGKB,2023-08-01 19:50,0.0325,51.3308,183.0,60.8,55.4,82.37,210.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.8,EGKB 011950Z 21010KT 9999 FEW015 16/13 Q1001,NaN
2038808,EGKB,2023-08-01 20:20,0.0325,51.3308,183.0,59.0,55.4,87.83,200.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,EGKB 012020Z 20011KT 9999 FEW025 15/13 Q1001,NaN
2038809,EGKB,2023-08-01 20:50,0.0325,51.3308,183.0,59.0,55.4,87.83,210.0,11.0,...,RA,NaN,NaN,NaN,NaN,NaN,NaN,59.0,EGKB 012050Z 21011KT 9999 RA SCT050 15/13 Q1001,NaN


In [53]:
processed_df = process_asos_rawdata(raw_df)

In [54]:
processed_df

,station,lon,lat,elevation,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,feel,date,time
0,EGUN,0.4864,52.3619,10.0,285.2,285.2,100.00,210.0,4.629996,0.0,0.764032,101890.0,6.21,NaN,285.2,20220101,00:26
1,EGUN,0.4864,52.3619,10.0,285.4,285.4,100.00,210.0,5.144440,0.0,0.764032,101890.0,6.21,NaN,285.4,20220101,00:56
2,EGUN,0.4864,52.3619,10.0,286.2,286.2,100.00,200.0,4.629996,0.0,0.763778,101860.0,6.21,NaN,286.2,20220101,01:26
3,EGUN,0.4864,52.3619,10.0,285.8,285.8,100.00,210.0,5.658884,0.0,0.764032,101890.0,6.21,NaN,285.8,20220101,01:56
4,EGUN,0.4864,52.3619,10.0,286.3,286.3,100.00,200.0,4.629996,0.0,0.763778,101860.0,6.21,NaN,286.3,20220101,02:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,0.0325,51.3308,183.0,289.2,287.2,87.92,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,289.2,20230801,19:20
2038807,EGKB,0.0325,51.3308,183.0,289.2,286.2,82.37,210.0,5.144440,0.0,0.750824,NaN,6.21,NaN,289.2,20230801,19:50
2038808,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,200.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,20230801,20:20
2038809,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,20230801,20:50


In [64]:
def time_select(processed_df, start_date, end_date):
    processed_df["DATE"] = pd.to_datetime(processed_df["DATE"], format="%Y%m%d")

    filtered_df = processed_df[
        (processed_df["DATE"] >= start_date) & (processed_df["DATE"] <= end_date)
    ]

    return filtered_df

In [67]:
start_date = pd.Timestamp("2022-08-01")
end_date = pd.Timestamp("2023-08-01")
processed_df_2 = time_utils.time_select(processed_df, "date", start_date, end_date)

In [68]:
processed_df_2

,station,lon,lat,elevation,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,feel,date,time
6642,EGUN,0.4864,52.3619,10.0,286.2,286.2,100.00,220.0,1.028888,0.0,0.763778,101860.0,6.21,NaN,286.2,2022-08-01,00:46
6643,EGUN,0.4864,52.3619,10.0,286.6,286.6,100.00,0.0,0.000000,0.0,0.763778,101860.0,6.21,NaN,286.6,2022-08-01,00:56
6644,EGUN,0.4864,52.3619,10.0,287.4,287.4,100.00,0.0,0.000000,0.0,0.764032,101890.0,5.59,NaN,287.4,2022-08-01,01:56
6645,EGUN,0.4864,52.3619,10.0,286.2,286.2,100.00,230.0,1.543332,0.0,0.764032,101890.0,5.59,NaN,286.2,2022-08-01,02:30
6646,EGUN,0.4864,52.3619,10.0,286.2,286.2,100.00,0.0,0.000000,0.0,0.763778,101860.0,6.21,NaN,286.2,2022-08-01,02:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,0.0325,51.3308,183.0,289.2,287.2,87.92,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,289.2,2023-08-01,19:20
2038807,EGKB,0.0325,51.3308,183.0,289.2,286.2,82.37,210.0,5.144440,0.0,0.750824,NaN,6.21,NaN,289.2,2023-08-01,19:50
2038808,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,200.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,2023-08-01,20:20
2038809,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,2023-08-01,20:50


In [79]:
columns_to_drop = [
    'lon',
    'lat',
    'elevation',
]
processed_df_2.drop(columns=columns_to_drop, inplace=True, errors='ignore')
processed_df_2

/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/3438336287.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df_2.drop(columns=columns_to_drop, inplace=True, errors='ignore')


,station,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,feel,date,time
6642,EGUN,286.2,286.2,100.00,220.0,1.028888,0.0,0.763778,101860.0,6.21,NaN,286.2,2022-08-01,00:46
6643,EGUN,286.6,286.6,100.00,0.0,0.000000,0.0,0.763778,101860.0,6.21,NaN,286.6,2022-08-01,00:56
6644,EGUN,287.4,287.4,100.00,0.0,0.000000,0.0,0.764032,101890.0,5.59,NaN,287.4,2022-08-01,01:56
6645,EGUN,286.2,286.2,100.00,230.0,1.543332,0.0,0.764032,101890.0,5.59,NaN,286.2,2022-08-01,02:30
6646,EGUN,286.2,286.2,100.00,0.0,0.000000,0.0,0.763778,101860.0,6.21,NaN,286.2,2022-08-01,02:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,289.2,287.2,87.92,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,289.2,2023-08-01,19:20
2038807,EGKB,289.2,286.2,82.37,210.0,5.144440,0.0,0.750824,NaN,6.21,NaN,289.2,2023-08-01,19:50
2038808,EGKB,288.2,286.2,87.83,200.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,2023-08-01,20:20
2038809,EGKB,288.2,286.2,87.83,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,2023-08-01,20:50


In [80]:
save_asos_processed_data(
    processed_df_2, country, data_folder, data_save_category, output_folder
)

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/processed_data/ASOS_DATA/GB_ASOS_DATA' created successfully.
GB_ASOS_processed_data.csv done!


In [81]:
def merge_csv_station(country, data_folder, data_category, output_folder):
    # Find csvs
    input_folder = folder_utils.find_folder(
        country, data_folder, data_category, output_folder
    )
    station_network_csv = "GB__asos_station_network.csv"
    asos_data_csv = "GB_ASOS_processed_data.csv"
    station_network_csv_path = os.path.join(input_folder, station_network_csv)
    asos_data_csv_path = os.path.join(input_folder, asos_data_csv)

    # Read csvs
    station_id_df = pd.read_csv(station_network_csv_path)
    station_info_df = pd.read_csv(asos_data_csv_path)

    columns_to_drop = [
        "Country",
        "Network",
        "Archive_Begin",
        "Archive_End",
        "Time_Domain",
    ]

    station_id_df = station_id_df.drop(columns=columns_to_drop)
    station_id_df.rename(columns={'ID': 'station'}, inplace=True)
    

    # Merge by "STATION"
    merged_df = pd.merge(station_id_df, station_info_df, on="station", how="left")

    return merged_df


In [82]:
# Example usage

country = "GB"
data_folder = "data"
data_test_category = "test_data"
data_read_category = "raw_data"
data_save_category = "processed_data"
output_folder = "ASOS_DATA"
merge_df = merge_csv_station(country, data_folder, data_save_category, output_folder)
merge_df

,Name,station,Latitude,Logitude,Elevation,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,feel,date,time
0,Aberdeen,EGPD,57.2049,-2.2053,65.000000,286.2,283.2,81.99,190.0,3.086664,0.0,0.763524,NaN,6.21,NaN,286.2,2022-08-01,00:20
1,Aberdeen,EGPD,57.2049,-2.2053,65.000000,285.2,283.2,87.56,170.0,2.057776,0.0,0.763524,NaN,6.21,NaN,285.2,2022-08-01,00:50
2,Aberdeen,EGPD,57.2049,-2.2053,65.000000,284.2,282.2,87.47,170.0,2.572220,0.0,0.762762,NaN,6.21,NaN,284.2,2022-08-01,01:20
3,Aberdeen,EGPD,57.2049,-2.2053,65.000000,283.2,282.2,93.50,180.0,2.057776,0.0,0.762762,NaN,6.21,NaN,283.2,2022-08-01,01:50
4,Aberdeen,EGPD,57.2049,-2.2053,65.000000,283.2,282.2,93.50,160.0,1.543332,0.0,0.762762,NaN,6.21,NaN,283.2,2022-08-01,02:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343714,Yorkshire and Humberside,EGSY,53.3943,-1.3885,68.423706,288.2,287.2,93.74,120.0,3.601108,0.0,0.745490,NaN,6.21,NaN,288.2,2023-08-01,21:50
1343715,Yorkshire and Humberside,EGSY,53.3943,-1.3885,68.423706,288.2,287.2,93.74,120.0,5.144440,0.0,0.745490,NaN,6.21,NaN,288.2,2023-08-01,22:20
1343716,Yorkshire and Humberside,EGSY,53.3943,-1.3885,68.423706,288.2,287.2,93.74,120.0,5.658884,0.0,0.744728,NaN,6.21,NaN,288.2,2023-08-01,22:50
1343717,Yorkshire and Humberside,EGSY,53.3943,-1.3885,68.423706,288.2,287.2,93.74,110.0,4.629996,0.0,0.743966,NaN,2.49,NaN,288.2,2023-08-01,23:20


---

---
